## 1. The oldest businesses in the world
What characteristics enable a business to stand the test of time? To help answer this question, BusinessFinancing.co.uk <a href="https://businessfinancing.co.uk/the-oldest-company-in-almost-every-country">researched</a> the oldest company that is still in business in almost every country and compiled the results into a dataset. Let's explore this work to better understand these historic businesses.

In [21]:
import pandas as pd

businesses = pd.read_csv('businesses.csv')

sorted_businesses = businesses.sort_values('year_founded')

sorted_businesses.head()

,business,year_founded,category_code,country_code
64,Kongō Gumi,578,CAT6,JPN
94,St. Peter Stifts Kulinarium,803,CAT4,AUT
107,Staffelter Hof Winery,862,CAT9,DEU
106,Monnaie de Paris,864,CAT12,FRA
103,The Royal Mint,886,CAT12,GBR


## 2. The oldest businesses in North America
<p>So far we've learned that Kongō Gumi is the world's oldest continuously operating business, beating out the second oldest business by well over 100 years! It's a little hard to read the country codes, though. Wouldn't it be nice if we had a list of country names to go along with the country codes? Having useful information in different files is a common problem: for data storage, it's better to keep different types of data separate, but for analysis, we want all the data in one place. To solve this, we'll have to join the two tables together.

<p>Since <code>countries.csv</code> contains a <code>continent</code> column, merging the datasets will also allow us to look at the oldest business on each continent! </p>

In [11]:
countries = pd.read_csv('countries.csv')

businesses_countries = sorted_businesses.merge(countries, on="country_code")

north_america = businesses_countries[businesses_countries['continent'] == 'North America']
north_america.head()

,business,year_founded,category_code,country_code,country,continent
22,La Casa de Moneda de México,1534,CAT12,MEX,Mexico,North America
28,Shirley Plantation,1638,CAT1,USA,United States,North America
33,Hudson's Bay Company,1670,CAT17,CAN,Canada,North America
35,Mount Gay Rum,1703,CAT9,BRB,Barbados,North America
40,Rose Hall,1770,CAT19,JAM,Jamaica,North America


## 3. The oldest business on each continent
<p>Now we can see that the oldest company in North America is La Casa de Moneda de México, founded in 1534. Why stop there, though, when we could easily find out the oldest business on every continent? </p>

In [12]:
continent = businesses_countries.groupby('continent').agg({'year_founded':'min'})

merged_continent = continent.merge(businesses_countries, on=['continent','year_founded'])

subset_merged_continent = merged_continent[['continent','country','business','year_founded']]
subset_merged_continent

,continent,country,business,year_founded
0,Africa,Mauritius,Mauritius Post,1772
1,Asia,Japan,Kongō Gumi,578
2,Europe,Austria,St. Peter Stifts Kulinarium,803
3,North America,Mexico,La Casa de Moneda de México,1534
4,Oceania,Australia,Australia Post,1809
5,South America,Peru,Casa Nacional de Moneda,1565


## 4. Unknown oldest businesses
<p>BusinessFinancing.co.uk wasn't able to determine the oldest business for some countries, and those countries are simply left off of <code>businesses.csv</code> and, by extension, <code>businesses</code>. However, the <code>countries</code> that we created <em>does</em> include all countries in the world, regardless of whether the oldest business is known. </p>
<p>We can compare the two datasets in one DataFrame to find out which countries don't have a known oldest business! </p>

In [13]:
all_countries = businesses.merge(countries, on='country_code',how='right', indicator=True)

missing_countries = all_countries[all_countries['_merge']!='both']

missing_countries_series = missing_countries['country']

missing_countries_series

1                                Angola
7                   Antigua and Barbuda
18                              Bahamas
48                   Dominican Republic
50                              Ecuador
57                                 Fiji
59      Micronesia, Federated States of
63                                Ghana
65                               Gambia
69                              Grenada
79            Iran, Islamic Republic of
89                           Kyrgyzstan
91                             Kiribati
92                Saint Kitts and Nevis
107                              Monaco
108                Moldova, Republic of
110                            Maldives
112                    Marshall Islands
131                               Nauru
138                               Palau
139                    Papua New Guinea
143                            Paraguay
144                 Palestine, State of
153                     Solomon Islands
160                            Suriname


In [15]:
new_businesses = pd.read_csv("new_businesses.csv")

all_businesses = pd.concat([new_businesses,businesses])

new_all_countries = all_businesses.merge(countries, on='country_code', how='outer', indicator=True)
new_missing_countries = new_all_countries[new_all_countries['_merge']!='both']

count_missing = new_missing_countries.groupby('continent').agg({'country':'count'})
count_missing.columns = ['count_missing']
count_missing

,count_missing
continent,
Africa,3
Asia,7
Europe,2
North America,5
Oceania,10
South America,3


## 6. The oldest industries
Remember our oldest business in the world, Kongō Gumi? We know Kongō Gumi was founded in the year 578 in Japan, but it's a little hard to decipher which industry it's in. Information about what the <code>category_code</code> column refers to is in <code>"datasets/categories.csv"</code>: </p>

<p>Let's use <code>categories.csv</code> to understand how many oldest businesses are in each category of industry.</p>

In [17]:
categories = pd.read_csv("categories.csv")
businesses_categories = businesses.merge(categories, on='category_code')

count_business_cats = businesses_categories.groupby('category').agg({'business':'count'})

years_business_cats = businesses_categories.groupby("category").agg({'year_founded':'sum'})

count_business_cats.columns = ['count']
years_business_cats.columns = ['total_years_in_business']
display(count_business_cats.head(), years_business_cats.head())

,count
category,
Agriculture,6
Aviation & Transport,19
Banking & Finance,37
"Cafés, Restaurants & Bars",6
Conglomerate,3


,total_years_in_business
category,
Agriculture,10669
Aviation & Transport,36598
Banking & Finance,70302
"Cafés, Restaurants & Bars",8532
Conglomerate,5671


## 7. Restaurant representation
<p>No matter how we measure it, looks like Banking and Finance is an excellent industry to be in if longevity is our goal! Let's zoom in on another industry: cafés, restaurants, and bars. Which restaurants in our dataset have been around since before the year 1800?</p>

In [18]:
old_restaurants = businesses_categories.query('category_code=="CAT4" and year_founded<1800')

old_restaurants = old_restaurants.sort_values('year_founded', ascending=True)
old_restaurants

,business,year_founded,category_code,country_code,category
142,St. Peter Stifts Kulinarium,803,CAT4,AUT,"Cafés, Restaurants & Bars"
143,Sean's Bar,900,CAT4,IRL,"Cafés, Restaurants & Bars"
139,Ma Yu Ching's Bucket Chicken House,1153,CAT4,CHN,"Cafés, Restaurants & Bars"


## 8. Categories and continents
<p>St. Peter Stifts Kulinarium is old enough that the restaurant is believed to have served Mozart - and it would have been over 900 years old even when he was a patron! Let's finish by looking at the oldest business in each category of commerce for each continent. </p>

In [22]:
businesses_categories_countries = businesses_categories.merge(countries, on='country_code')

businesses_categories_countries = businesses_categories_countries.sort_values('year_founded')

oldest_by_continent_category = businesses_categories_countries.groupby(['continent','category']).agg({'year_founded':'min'})
oldest_by_continent_category.head()

year_founded
continent category                                       
Africa    Agriculture                                1947
          Aviation & Transport                       1854
          Banking & Finance                          1892
          Distillers, Vintners, & Breweries          1933
          Energy                                     1968